# Advent of Code 2021 - Day 19

In [457]:
data class Rotation(
    val x1: Int, val y1: Int, val z1: Int,
    val x2: Int, val y2: Int, val z2: Int,
    val x3: Int, val y3: Int, val z3: Int,
)

object Rotations : Set<Rotation> by setOf(
    Rotation(1,0,0, 0,1,0, 0,0,1),
    Rotation(-1,0,0, 0,-1,0, 0,0,1),
    Rotation(-1,0,0, 0,1,0, 0,0,-1),
    Rotation(1,0,0, 0,-1,0, 0,0,-1),

    Rotation(0,0,-1, 0,-1,0, -1,0,0),
    Rotation(0,0,1, 0,1,0, -1,0,0),
    Rotation(0,0,1, 0,-1,0, 1,0,0),
    Rotation(0,0,-1, 0,1,0, 1,0,0),

    Rotation(0,1,0, 0,0,1, 1,0,0),
    Rotation(0,-1,0, 0,0,1, -1,0,0),
    Rotation(0,1,0, 0,0,-1, -1,0,0),
    Rotation(0,-1,0, 0,0,-1, 1,0,0),

    Rotation(0,-1,0, -1,0,0, 0,0,-1),
    Rotation(0,1,0, -1,0,0, 0,0,1),
    Rotation(0,-1,0, 1,0,0, 0,0,1),
    Rotation(0,1,0, 1,0,0, 0,0,-1),

    Rotation(0,0,1, 1,0,0, 0,1,0),
    Rotation(0,0,1, -1,0,0, 0,-1,0),
    Rotation(0,0,-1, -1,0,0, 0,1,0),
    Rotation(0,0,-1, 1,0,0, 0,-1,0),

    Rotation(-1,0,0, 0,0,-1, 0,-1,0),
    Rotation(-1,0,0, 0,0,1, 0,1,0),
    Rotation(1,0,0, 0,0,1, 0,-1,0),
    Rotation(1,0,0, 0,0,-1, 0,1,0),
)

data class Beacon(val x: Int, val y: Int, val z: Int)

fun Beacon.rotateBy(rotation: Rotation) = Beacon(
    x = (rotation.x1 * x) + (rotation.y1 * y) + (rotation.z1 * z),
    y = (rotation.x2 * x) + (rotation.y2 * y) + (rotation.z2 * z),
    z = (rotation.x3 * x) + (rotation.y3 * y) + (rotation.z3 * z),
)

fun Beacon.rotateBy(rotations: Collection<Rotation>) = rotations.map { rotateBy(it) }

data class Scanner(val beacons: Set<Beacon>)

In [458]:
import java.io.File
import java.util.Scanner
import kotlin.sequences.generateSequence

val regex = Regex("""(-?\d+),(-?\d+),(-?\d+)""")

val scanners: List<Scanner> = Scanner(File("Day19.input.txt"))
    .apply { useDelimiter("${System.lineSeparator()}${System.lineSeparator()}") }
    .use { scanner ->
        generateSequence { if(scanner.hasNext()) scanner.next() else null }
            .map {
                it.lineSequence().drop(1)
                    .map { regex.find(it)!!.destructured }
                    .map { (x, y, z) -> Beacon(x = x.toInt(), y = y.toInt(), z = z.toInt()) }
            }
            .map { Scanner(it.toSet()) }
            .toList()
    }

## Part 1

Given a list of `scanners` find all of the unique `Beacon`s. Each `Scanner` can be randomly orientated in 90 degree increments. Two scanners overlap when at least 12 of their beacons overlap.

In [459]:
import kotlin.collections.ArrayDeque

val MIN_MATCHING = 12

inline fun <A,B> Iterable<A>.mapToSet(fn: (A) -> B): Set<B> = mapTo(mutableSetOf(), fn)

fun Scanner.rotateBy(rotation: Rotation) = Scanner(beacons.mapToSet { it.rotateBy(rotation) })
fun Scanner.rotateBy(rotations: Collection<Rotation>): List<Scanner> = rotations.map { rotateBy(it) }

data class Translation(val dx: Int, val dy: Int, val dz: Int)
fun Beacon.translateBy(translation: Translation) = Beacon(x = x + translation.dx, y = y + translation.dy, z = z + translation.dz)
fun Scanner.translateBy(translation: Translation) = Scanner(beacons.mapToSet { it.translateBy(translation) })

operator fun Beacon.minus(other: Beacon) = Translation(dx = x - other.x, dy = y - other.y, dz = z - other.z)

fun Scanner.overlaps(other: Scanner): Int = other.beacons.count { it in beacons }

operator fun Scanner.plus(other: Scanner) = Scanner(beacons + other.beacons)

fun Scanner.correlate(other: Scanner): Pair<Scanner, Translation>? {
    other.rotateBy(Rotations).forEach { rotated ->
        beacons.flatMap { beacon -> rotated.beacons.map { beacon - it } }
            .forEach { translation ->
                val translated = rotated.translateBy(translation)
                if (overlaps(translated) >= MIN_MATCHING)
                    return (this + translated) to translation
            }
    }
    return null
}

tailrec fun Scanner.correlateAll(other: List<Scanner>, translations: List<Translation> = listOf()): Pair<Scanner, List<Translation>> =
    if (other.isEmpty()) this to translations
    else other.first().let {
        correlate(it)
            ?.let { it.first.correlateAll(other.drop(1), translations + it.second) } 
            ?: correlateAll(other.drop(1) + it, translations)
    }

val (scanner, translations) = scanners[0].correlateAll(scanners.drop(1))
scanner.beacons.size

483

### Notes

Since I took a computer graphics course in college, the first thought here was matrix operations for rotating and translating. Then I broke down each operation piece by piece. The core logic here is to anchor yourself against a single "correct" orientation (any one orientation is equally valid), so we can just choose the first one. To find if two `Scanner`s overlap, we need to first check all rotational orientations. Within each rotation, we translate each `Beacon` to each "anchor" `Beacon` and then count all of the matches. If we have twelve or more then we can combine all of the `Beacon`s into a single `Scanner`. We can then just repeat this process until we have a single fully merged `Scanner`.

I think I ended up being pretty happy with the code, however the execution ended up being quite slow (22s on my MacBook Pro). Without a significant deviation from the core logic, a big performance benefit would be to utilize more mutations rather than having each operation return new objects.

## Part 2

Calculate the largest `Manhattan Distance` between all of the `Scanner`s.

In [460]:
fun Translation.manhattanDistance(other: Translation): Int = abs(dx - other.dx) + abs(dy - other.dy) + abs(dz - other.dz)

translations.maxOfOrNull { outer ->
    translations.maxOfOrNull { outer.manhattanDistance(it) } ?: 0
}

14804

### Notes

I actually had already implemented `Translation` for part 1, so not much needed to change. Since we were translating each `Scanner` to the first `Scanner` as "0,0" we can just treat each translation as the location of a `Scanner`. I had to modify the `Scanner::correlate` and `Scanner::correlateAll` functions to return the `Translation` as well as the `Scanner`. I think this makes the API messier, but I didn't want to put more effort into this problem.